# link to Mongodb

In [5]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient()
db = client.fashion  

In [33]:
#db.create_collection('product')

#db.drop_collection('product')

{'nIndexesWas': 1, 'ns': 'fashion.product', 'ok': 1.0}

In [6]:
db.collection_names()

<ipython-input-6-8ff4d2505e25>:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  db.collection_names()


['test_run', 'product', 'test_run_2', 'test', 'test_1', 'test_run_1']

In [8]:
#db.drop_collection('XXXX')

# Get my api

In [7]:
import api
import datetime

date = datetime.datetime.today().strftime('%m-%d-%y')

api = api.Api()
total_pages = api.get_listings()['listingPagination']['totalPages']     

### 1. [main.py]  web scrap for ALL total pages
function name : run_get_pages()

In [36]:
#for page in range(1, 3 + 1):

for page in range(1, total_pages + 1):
    print(page)
    ff_json = api.get_listings(page=page)
    
    for i in range(len(ff_json['listingItems']['items'])):
        product = ff_json['listingItems']['items'][i]
        
        db.product.insert_one(product)                # 4776 total pages

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


### 2. [main.py]  rescrap to refresh new items for UPDATE
function name: run_get_new()

   - first page only 
   - cron job for weekly 

In [9]:
ff_json_new = api.get_listings()  #only first page

database_id = [dict_item['id'] for dict_item in list(db.product.find({},{'_id':0, 'id':1}))]

new_item = 0

for item in ff_json_new['listingItems']['items']:
    #print(item)
    #print(item['id'])
    
    if item['id'] not in database_id:
        new_item += 1
        db.product.insert_one(item)
    #else:
        #print("nothing new")
if new_item == 0:
    print("nothing new")

nothing new


### 3. check if db looks ok

In [10]:
cursor = db.product.find({}, {'_id':0, 'id':1, 'shortDescription':1, 'images.cutOut':1, 'brand':1, 'priceInfo.formattedFinalPrice':1, 'url':1}) 
#list(cursor)

In [36]:
# count
db.product.count_documents({})   

428940


### 4. find if any duplicates and delete those (!^^")

In [37]:
pipeline = [                                                  # TAQ: how to delete duplicates
    {'$group': {'_id': '$id', 'count': {'$sum': 1} }},                                          #, 'total': {'$sum': '$id.count'}
    {'$match': {'count': {'$gt': 1}}},
    {'$project': {'id': '$_id', 'count': 1}},                                                   # , 'total':1
    {'$sort': {'count': -1}}
]

list(db.product.aggregate(pipeline))

[{'_id': 14962673, 'count': 3, 'id': 14962673},
 {'_id': 16043555, 'count': 3, 'id': 16043555},
 {'_id': 16036158, 'count': 3, 'id': 16036158},
 {'_id': 15814158, 'count': 3, 'id': 15814158},
 {'_id': 15308883, 'count': 3, 'id': 15308883},
 {'_id': 15465446, 'count': 3, 'id': 15465446},
 {'_id': 16984353, 'count': 3, 'id': 16984353},
 {'_id': 14133675, 'count': 3, 'id': 14133675},
 {'_id': 16383515, 'count': 3, 'id': 16383515},
 {'_id': 16919244, 'count': 3, 'id': 16919244},
 {'_id': 14024933, 'count': 3, 'id': 14024933},
 {'_id': 16573514, 'count': 3, 'id': 16573514},
 {'_id': 16565637, 'count': 3, 'id': 16565637},
 {'_id': 12228537, 'count': 3, 'id': 12228537},
 {'_id': 17034730, 'count': 3, 'id': 17034730},
 {'_id': 16018057, 'count': 3, 'id': 16018057},
 {'_id': 16702408, 'count': 3, 'id': 16702408},
 {'_id': 16462707, 'count': 3, 'id': 16462707},
 {'_id': 16255480, 'count': 3, 'id': 16255480},
 {'_id': 13678571, 'count': 3, 'id': 13678571},
 {'_id': 17059781, 'count': 3, 'id': 170

In [46]:
c = db.product.find({'id': 16036158}, {'_id':0, 'id':1, 'shortDescription':1, 'images.cutOut':1, 'brand':1, 'priceInfo.formattedFinalPrice':1, 'url':1})
list(c)

[{'id': 16036158,
  'shortDescription': 'mesh-panel low-top sneakers',
  'brand': {'id': 17886, 'name': 'Y-3'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/03/61/58/16036158_29977860_480.jpg'},
  'priceInfo': {'formattedFinalPrice': '$617'},
  'url': '/ca/shopping/men/y-3-mesh-panel-low-top-sneakers-item-16036158.aspx?storeid=10646'},
 {'id': 16036158,
  'shortDescription': 'mesh-panel low-top sneakers',
  'brand': {'id': 17886, 'name': 'Y-3'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/03/61/58/16036158_29977860_480.jpg'},
  'priceInfo': {'formattedFinalPrice': '$617'},
  'url': '/ca/shopping/men/y-3-mesh-panel-low-top-sneakers-item-16036158.aspx?storeid=10646'},
 {'id': 16036158,
  'shortDescription': 'mesh-panel low-top sneakers',
  'brand': {'id': 17886, 'name': 'Y-3'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/03/61/58/16036158_29977860_480.jpg'},
  'priceInfo': {'formattedFinalPrice': '$617'},
  'url': '/c

In [41]:
c = db.product.find({'id': 14910454}, {'_id':0, 'id':1, 'shortDescription':1, 'images.cutOut':1, 'brand':1, 'priceInfo.formattedFinalPrice':1, 'url':1})
list(c)

[{'id': 14910454,
  'shortDescription': 'Grecca border baroque stripe leggings',
  'brand': {'id': 6803, 'name': 'Versace'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/14/91/04/54/14910454_24521975_480.jpg'},
  'priceInfo': {'formattedFinalPrice': '$275'},
  'url': '/ca/shopping/women/versace-grecca-border-baroque-stripe-leggings-item-14910454.aspx?storeid=13537'}]

## Decide what catergories I want -> filter catergories
- bag  (my mainly focus this time)
- sneakers


In [6]:
# bag

cursor = db.product.find({'shortDescription': {'$regex' : "bag"}}, {'_id':0, 'id':1, 'shortDescription':1, 'images.cutOut':1})  #, 'brand':1, 'url':1})  #'_id':0, 'items': 1
#list(cursor)

## Load image via MongoDB
download via url into local folder

In [3]:
import shutil

import requests

### BAG

In [15]:
cursor = db.product.find({'shortDescription': {'$regex' : "bag"}}, {'_id':0, 'id':1 ,'brand':1, 'images.cutOut':1})
#list(cursor)

In [18]:
pipeline = [
    {'$match': {'shortDescription': {'$regex' : "bag"}}},                  
    {'$group': {'_id': '$bag', 'total_bags': {'$sum': 1}}},        
    {'$project': {'_id': 0, 'total_bags': 1}},   # , 'bag': '$_id'                                          
]

list(db.product.aggregate(pipeline))  

[{'total_bags': 21516}]

##### (eda) Count each bag brand numbers 

In [20]:
# count bag's brand numbers

pipeline = [
    {'$match': {'shortDescription': {'$regex' : "bag"}}},                   # filter
    {'$group': {'_id': '$brand.name', 'num_brands': {'$sum': 1}}},         # group by brand name, add 1 for every record and store i num_restaurants
    {'$project': {'_id': 0, 'brand': '$_id', 'num_brands': 1}},            # rename _id to brand
    {'$sort': {'num_brands': -1}}    # -1                                  # sort by num_brand, ascending
]

# len(list(db.product.aggregate(pipeline)))
list(db.product.aggregate(pipeline))

[{'num_brands': 701, 'brand': 'Bottega Veneta'},
 {'num_brands': 603, 'brand': 'Marni'},
 {'num_brands': 579, 'brand': 'Saint Laurent'},
 {'num_brands': 555, 'brand': 'Dolce & Gabbana'},
 {'num_brands': 547, 'brand': 'Valentino Garavani'},
 {'num_brands': 484, 'brand': 'Chanel Pre-Owned'},
 {'num_brands': 426, 'brand': 'Michael Michael Kors'},
 {'num_brands': 384, 'brand': 'Alexander McQueen'},
 {'num_brands': 355, 'brand': 'Moschino'},
 {'num_brands': 349, 'brand': 'Tory Burch'},
 {'num_brands': 348, 'brand': 'Jil Sander'},
 {'num_brands': 340, 'brand': 'Furla'},
 {'num_brands': 329, 'brand': 'Fendi'},
 {'num_brands': 313, 'brand': 'Stella McCartney'},
 {'num_brands': 305, 'brand': 'Karl Lagerfeld'},
 {'num_brands': 297, 'brand': 'Marc Jacobs'},
 {'num_brands': 296, 'brand': 'Salvatore Ferragamo'},
 {'num_brands': 295, 'brand': 'Hermès'},
 {'num_brands': 281, 'brand': 'Givenchy'},
 {'num_brands': 260, 'brand': 'Jimmy Choo'},
 {'num_brands': 258, 'brand': 'Love Moschino'},
 {'num_brand

## Download img by each brand 

### 1. YSL

In [52]:
# YSL

cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Saint Laurent'}}, {'_id':0, 'id':1 , 'brand.name':1, 'images.cutOut':1})  # 'id':1 , 'shortDescription':1, 'brand':1, 'url':1})  #'_id':0, 'items': 1

list(cursor)

[{'id': 15535296,
  'brand': {'name': 'Saint Laurent'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/15/53/52/96/15535296_32422863_480.jpg'}},
 {'id': 16080671,
  'brand': {'name': 'Saint Laurent'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/08/06/71/16080671_30245929_480.jpg'}},
 {'id': 12449725,
  'brand': {'name': 'Saint Laurent'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/12/44/97/25/12449725_11500128_480.jpg'}},
 {'id': 14574977,
  'brand': {'name': 'Saint Laurent'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/14/57/49/77/14574977_23466270_480.jpg'}},
 {'id': 16363460,
  'brand': {'name': 'Saint Laurent'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/36/34/60/16363460_33947908_480.jpg'}},
 {'id': 12561047,
  'brand': {'name': 'Saint Laurent'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/12/56/10/47/12561047_30537144_480.jpg'}},
 {'id': 14037338,
  'b

In [29]:
# YSL

cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Saint Laurent'}}, {'_id':0, 'id':1 , 'images.cutOut':1})  # 'id':1 , 'shortDescription':1, 'brand':1, 'url':1})  #'_id':0, 'items': 1

k = 0
for item in cursor:
    url = item['images']['cutOut']
    
    response = requests.get(url, stream=True)
    
    label = 'bag_YSL_'
    
    with open( f'{label}/img{k}.png', 'wb') as out_file:   
        shutil.copyfileobj(response.raw, out_file)   
        
    k += 1   
                                             

### 2. Gucci


In [12]:
#'Gucci'

In [30]:
cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Gucci'}}, {'_id':0, 'id':1 , 'images.cutOut':1})  # 'id':1 , 'shortDescription':1, 'brand':1, 'url':1})  #'_id':0, 'items': 1

k = 0
for item in cursor:
    url = item['images']['cutOut']
    
    response = requests.get(url, stream=True)
    
    label = 'bag_Gucci_'
    
    with open( f'{label}/img{k}.png', 'wb') as out_file:   
        shutil.copyfileobj(response.raw, out_file)   
        
    k += 1   
           

### 3. Prada

In [ ]:
# 'Prada'

In [31]:
cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Prada'}}, {'_id':0, 'id':1 , 'images.cutOut':1})  # 'id':1 , 'shortDescription':1, 'brand':1, 'url':1})  #'_id':0, 'items': 1

k = 0
for item in cursor:
    url = item['images']['cutOut']
    
    response = requests.get(url, stream=True)
    
    label = 'bag_Prada_'
    
    with open( f'{label}/img{k}.png', 'wb') as out_file:   
        shutil.copyfileobj(response.raw, out_file)   
        
    k += 1   
           

### 4. Hermès

In [60]:
# 'Hermès'
cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Hermès'}}, {'_id':0, 'id':1 , 'brand.name':1, 'images.cutOut':1})  # 'id':1 , 'shortDescription':1, 'brand':1, 'url':1})  #'_id':0, 'items': 1

list(cursor)


[{'id': 16255102,
  'brand': {'name': 'Hermès'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/25/51/02/16255102_31075326_480.jpg'}},
 {'id': 13366252,
  'brand': {'name': 'Hermès'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/13/36/62/52/13366252_15447544_480.jpg'}},
 {'id': 16827902,
  'brand': {'name': 'Hermès'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/82/79/02/16827902_33475171_480.jpg'}},
 {'id': 15523460,
  'brand': {'name': 'Hermès'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/15/52/34/60/15523460_28408170_480.jpg'}},
 {'id': 17036252,
  'brand': {'name': 'Hermès'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/17/03/62/52/17036252_34343444_480.jpg'}},
 {'id': 16509702,
  'brand': {'name': 'Hermès'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/50/97/02/16509702_32090225_480.jpg'}},
 {'id': 15662902,
  'brand': {'name': 'Hermès'},
  'images': {'c

In [32]:
cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Hermès'}}, {'_id':0, 'id':1 , 'images.cutOut':1})  

k = 0
for item in cursor:
    url = item['images']['cutOut']
    
    response = requests.get(url, stream=True)
    
    label = 'bag_Hermes_'
    
    with open( f'{label}/img{k}.png', 'wb') as out_file:   
        shutil.copyfileobj(response.raw, out_file)   
        
    k += 1   
   

### 5. Louis Vuitton

In [ ]:
# 'Louis Vuitton'

In [33]:
cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Louis Vuitton'}}, {'_id':0, 'id':1 , 'images.cutOut':1})  

k = 0
for item in cursor:
    url = item['images']['cutOut']
    
    response = requests.get(url, stream=True)
    
    label = 'bag_LV_'
    
    with open( f'{label}/img{k}.png', 'wb') as out_file:   
        shutil.copyfileobj(response.raw, out_file)   
        
    k += 1   

### 6. Chanel

In [ ]:
# 'Chanel Pre-Owned'

In [35]:
cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Chanel Pre-Owned'}}, {'_id':0, 'id':1 , 'images.cutOut':1})  

k = 0
for item in cursor:
    url = item['images']['cutOut']
    
    response = requests.get(url, stream=True)
    
    label = 'bag_Chanel_'
    
    with open( f'{label}/img{k}.png', 'wb') as out_file:   
        shutil.copyfileobj(response.raw, out_file)   
        
    k += 1  

## Optional 
Dolce & Gabbana
### 7. Fendi

In [24]:
cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Fendi'}}, {'_id':0, 'id':1 , 'brand.name':1, 'images.cutOut':1})  # 'id':1 , 'shortDescription':1, 'brand':1, 'url':1})  #'_id':0, 'items': 1

list(cursor)

[{'id': 16369034,
  'brand': {'name': 'Fendi'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/36/90/34/16369034_32284557_480.jpg'}},
 {'id': 16720984,
  'brand': {'name': 'Fendi'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/72/09/84/16720984_34641272_480.jpg'}},
 {'id': 16490024,
  'brand': {'name': 'Fendi'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/49/00/24/16490024_32054277_480.jpg'}},
 {'id': 12501660,
  'brand': {'name': 'Fendi'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/12/50/16/60/12501660_11684451_480.jpg'}},
 {'id': 15366560,
  'brand': {'name': 'Fendi'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/15/36/65/60/15366560_28560110_480.jpg'}},
 {'id': 15422160,
  'brand': {'name': 'Fendi'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/15/42/21/60/15422160_29245376_480.jpg'}},
 {'id': 16839352,
  'brand': {'name': 'Fendi'},
  'images': {'cutOut':

In [26]:
cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Fendi'}}, {'_id':0, 'id':1 , 'images.cutOut':1})  

k = 0
for item in cursor:
    url = item['images']['cutOut']
    
    response = requests.get(url, stream=True)
    
    label = 'bag_Fendi_'
    
    with open( f'{label}/img{k}.png', 'wb') as out_file:   
        shutil.copyfileobj(response.raw, out_file)   
        
    k += 1  


### 8. Balenciaga



In [25]:
cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Balenciaga'}}, {'_id':0, 'id':1 , 'brand.name':1, 'images.cutOut':1})  # 'id':1 , 'shortDescription':1, 'brand':1, 'url':1})  #'_id':0, 'items': 1

list(cursor)

[{'id': 14770010,
  'brand': {'name': 'Balenciaga'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/14/77/00/10/14770010_27861241_480.jpg'}},
 {'id': 15360010,
  'brand': {'name': 'Balenciaga'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/15/36/00/10/15360010_28407563_480.jpg'}},
 {'id': 14774202,
  'brand': {'name': 'Balenciaga'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/14/77/42/02/14774202_26402214_480.jpg'}},
 {'id': 16580702,
  'brand': {'name': 'Balenciaga'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/58/07/02/16580702_33752886_480.jpg'}},
 {'id': 15921110,
  'brand': {'name': 'Balenciaga'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/15/92/11/10/15921110_32017314_480.jpg'}},
 {'id': 16192952,
  'brand': {'name': 'Balenciaga'},
  'images': {'cutOut': 'https://cdn-images.farfetch-contents.com/16/19/29/52/16192952_32711958_480.jpg'}},
 {'id': 15216872,
  'brand': {'name': 'B

In [27]:
cursor = db.product.find({'shortDescription': {'$regex' : "bag"}, 'brand.name':{'$regex':'Balenciaga'}}, {'_id':0, 'id':1 , 'images.cutOut':1})  

k = 0
for item in cursor:
    url = item['images']['cutOut']
    
    response = requests.get(url, stream=True)
    
    label = 'bag_Balenciaga_'
    
    with open( f'{label}/img{k}.png', 'wb') as out_file:   
        shutil.copyfileobj(response.raw, out_file)   
        
    k += 1  